# Purpose

This notebook works through an example workflow of tracking Keras experiments using [MLFlow](https://www.mlflow.org).  

# Data

The CA housing data will be used for this example, which is a simple regressiont task. It will be loaded from the `sklearn` data loader.  I'll split off 20% into a test set and an additional 20% into a validation set.  Finally, I'll standardize the data using `StandardScaler` ahead of modeling.

In [1]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd

housing = fetch_california_housing()

X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target, test_size=0.2
)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, test_size=0.2
)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

pd.DataFrame(X_train, columns=housing.feature_names).describe()


,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
count,1.320900e+04,1.320900e+04,1.320900e+04,1.320900e+04,1.320900e+04,1.320900e+04,1.320900e+04,1.320900e+04
mean,3.553521e-15,1.258740e-16,-5.648193e-16,-7.127482e-15,-4.195801e-17,6.863900e-16,3.574365e-14,-1.464878e-13
std,1.000038e+00,1.000038e+00,1.000038e+00,1.000038e+00,1.000038e+00,1.000038e+00,1.000038e+00,1.000038e+00
min,-1.779689e+00,-2.192862e+00,-1.841598e+00,-1.438534e+00,-1.263190e+00,-3.395727e-01,-1.445590e+00,-2.358488e+00
25%,-6.883022e-01,-8.439976e-01,-3.997890e-01,-1.862247e-01,-5.677128e-01,-8.730434e-02,-7.943021e-01,-1.110999e+00
50%,-1.717385e-01,2.879702e-02,-8.497085e-02,-1.012380e-01,-2.297322e-01,-2.983685e-02,-6.443653e-01,5.356868e-01
75%,4.589652e-01,6.635568e-01,2.442975e-01,1.356624e-03,2.626018e-01,3.705159e-02,9.768266e-01,7.801947e-01
max,5.886135e+00,1.853731e+00,5.095891e+01,6.545067e+01,2.407471e+01,8.664295e+01,2.958804e+00,2.626478e+00


# Model

The model used for this example will be a wide and deep network with the following characteristics:
- a deep path with `n_hidden` hidden layers with `n_neurons` at each layer
- a wide path connecting all inputs to the output
- all layers are fully connected
- two outpus:
    - one from the deep path alone, fit to the target
    - one from the concatenaded wide and deep paths, fit to the target

This type of multi-output architecture is usually used as a regularization technique, but I'm simply employing it here so my example has more than one loss to simultaneously minimize.  This model is very similar to the regression example I used in my [intro to Keras](https://github.com/mcnewcp/book-geron-ml-sklearn-keras-tensorflow/blob/main/10-intro-ann-keras/10-intro-ann-keras.ipynb) notebook and from Chapter 10 of [Hands on ML](https://www.oreilly.com/library/view/hands-on-machine-learning/9781492032632/), so I won't explain the Keras code below.

The model build code should be functionalized so that the hyperparameters are generalized in the build and compile steps for integration into hyperparameter tuning.  I'm pulling out the following hyperparameters for tuning:
- `n_hidden`: number of hidden layers
- `n_neurons`: number of neurons per layer
- `activation`: activation funciton used in hidden layers

*Note*: I'm not tuning learning rate here.  In general I think it's best practice to choose a sufficiently low learning rate, high number of epochs, and use early stopping.  The goal of this stage of hyperparameter tuning is to simply identify promising model candidates.  Once promising candidates have been identified, the learning rate will be fine tuned.


In [2]:
import tensorflow as tf
from tensorflow import keras

print("tf version:", tf.__version__, ", keras version:", keras.__version__)



2023-02-15 10:05:31.142284: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


tf version: 2.11.0 , keras version: 2.11.0


In [21]:
def build_model(n_hidden=1, n_neurons=20, activation="relu"):
    inp = keras.layers.Input(shape=[8], name="input")  # input layer
    for layer in range(n_hidden):  # sequentially add hidden layers
        if layer == 0:
            hl = keras.layers.Dense(n_neurons, activation=activation)(inp)
        else:
            hl = keras.layers.Dense(n_neurons, activation=activation)(hl)
    concat = keras.layers.Concatenate()([hl, inp])  # concat deep and wide paths
    main_output = keras.layers.Dense(1, name="main_output")(concat)  # combined output
    aux_output = keras.layers.Dense(1, name="aux_output")(hl)  # deep output
    model = keras.Model(inputs=[inp], outputs=[main_output, aux_output])
    model.compile(
        loss={"main_output": "mse", "aux_output": "mse"},
        loss_weights={
            "main_output": 0.9,
            "aux_output": 0.1,
        },  # weighting heavily towards main output
        optimizer=keras.optimizers.SGD(learning_rate=1e-3),
    )
    return model


# MLFlow Experiment Tracking

[MLFLow](https://www.mlflow.org) is a full-featured end-to-end ML lifecycle management platform, but all I'll be using it for in this example is experiment tracking.  The [documentation on tracking](https://www.mlflow.org/docs/latest/tracking.html) is quite good and so I'm working primarily from that.  There is even an automatic logging submodule for Keras and Tensorflow, `mlflow.tensorflow.autolog()` which I will try out first. In addition, nearly anything can be logged manually, including categories of metrics, parameters, tags, and artifacts.  Artifacts can be nearly anything including plots or the modles themselves.

## Auto-Logging

First I'll give the auto logging a shot and see what it logs.  I've had issues with the auto logging submodule for scikit-learn because it simply logged too many parameters to be useful and after the experiment count reached a certain threshold, performance in the dashboard tool suffered.

**Note**: the auto-logging submodule only works for tensorflow versions 2.3.0 - 2.11.0, which I had to specify manually with `pip` as the `conda` installer chose a version outside of that range.

By default, on first execution, MLFlow creates the directory `./mlruns` on default and stores all experiment related information as individual files within.  Another option is to store the information as a SQLite database, or incorporate into Databricks. 

In [22]:
import mlflow

mlflow.set_experiment(experiment_name="auto-log")  # this will create ./mlruns
mlflow.tensorflow.autolog()  # turn on auto logging

model = build_model(n_hidden=2)

with mlflow.start_run(run_name="auto-log-1") as run:
    history = model.fit(
        X_train,
        {"main_output": y_train, "aux_output": y_train},
        epochs=250,
        validation_data=(X_valid, {"main_output": y_valid, "aux_output": y_valid}),
        callbacks=[
            keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
        ],
        verbose=0,
    )
    total_loss, main_loss, aux_loss = model.evaluate(
        X_test, {"main_output": y_test, "aux_output": y_test}
    )



1/1 [==============================] - 0s 74ms/step


2023/02/15 11:17:12 WARNING mlflow.tensorflow: Failed to infer model signature: Expected one of (pandas.DataFrame, numpy array, dictionary of (name -> numpy.ndarray), pyspark.sql.DataFrame) but got '<class 'list'>'
2023/02/15 11:17:12 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: /var/folders/m8/0_prp1tj41s9n5xm0bfqp6wm0000gn/T/tmpty29kudo/model/data/model/assets


INFO:tensorflow:Assets written to: /var/folders/m8/0_prp1tj41s9n5xm0bfqp6wm0000gn/T/tmpty29kudo/model/data/model/assets


129/129 [==============================] - 0s 2ms/step - loss: 0.3357 - main_output_loss: 0.3320 - aux_output_loss: 0.3696


## Local Server

To serve the exploration tool locally, you simply run the following in the command line:

```zsh
mlflow ui
```

By default, this will look for local logged files in `./mlruns` and it will launch on port 5000.  If you need to change the location use `--backend-store-uri` and if you need to specify the port use `-p`.

The UI provides a simple table comparison of all runs within an experiment so you can quickly check the parameters used for each run and corresponding metrics to choose promising model candidates.  

It looks like the auto logger logs a lot of useful information including many parameters inferred from the model, early stopping results, and learning curves for each loss (see below).  Unsurprisingly though it doesn't log anything about model architecture which will be important in my case since most of my tuning will involve architecture changes, so these will need to be logged manually.

![MLFLow UI Screenshot](images/mlflow_ui_sn.png)

## Manual Logging

It's quite easy to log additional hyperparameters via `mlflow.log_params()`.  It accepts a dictionary of parameters and logs them to the corresponding run, as long as it's called under `with mlflow.start_run():`.  This means I'll need to define my hyperparameters in a dictionary at the start of my run, which will also aid in integrating optimization later.  I'll modify the above workflow to include both of these changes below.

In [5]:
hyper_params = {"n_hidden": 1, "n_neurons": 20, "activation": "relu"}

mlflow.set_experiment(experiment_name="auto-and-manual")
mlflow.tensorflow.autolog()  # turn on auto logging

model = build_model(**hyper_params)  # names must match

with mlflow.start_run(run_name="run-1") as run:
    history = model.fit(
        X_train,
        {"main_output": y_train, "aux_output": y_train},
        epochs=250,
        validation_data=(X_valid, {"main_output": y_valid, "aux_output": y_valid}),
        callbacks=[
            keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
        ],
        verbose=0,
    )
    mlflow.log_params(hyper_params)  # log all hyperparams



2023/02/15 10:06:45 INFO mlflow.tracking.fluent: Experiment with name 'auto-and-manual' does not exist. Creating a new experiment.


1/1 [==============================] - 0s 56ms/step


2023/02/15 10:08:20 WARNING mlflow.tensorflow: Failed to infer model signature: Expected one of (pandas.DataFrame, numpy array, dictionary of (name -> numpy.ndarray), pyspark.sql.DataFrame) but got '<class 'list'>'
2023/02/15 10:08:20 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: /var/folders/m8/0_prp1tj41s9n5xm0bfqp6wm0000gn/T/tmp9gt000uv/model/data/model/assets


INFO:tensorflow:Assets written to: /var/folders/m8/0_prp1tj41s9n5xm0bfqp6wm0000gn/T/tmp9gt000uv/model/data/model/assets


In [6]:
hyper_params = {"n_hidden": 3, "n_neurons": 10, "activation": "relu"}
model = build_model(**hyper_params)  # names must match

with mlflow.start_run(run_name="run-2") as run:
    history = model.fit(
        X_train,
        {"main_output": y_train, "aux_output": y_train},
        epochs=250,
        validation_data=(X_valid, {"main_output": y_valid, "aux_output": y_valid}),
        callbacks=[
            keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
        ],
        verbose=0,
    )
    mlflow.log_params(hyper_params)  # log all hyperparams



1/1 [==============================] - 0s 66ms/step


2023/02/15 10:09:53 WARNING mlflow.tensorflow: Failed to infer model signature: Expected one of (pandas.DataFrame, numpy array, dictionary of (name -> numpy.ndarray), pyspark.sql.DataFrame) but got '<class 'list'>'
2023/02/15 10:09:53 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: /var/folders/m8/0_prp1tj41s9n5xm0bfqp6wm0000gn/T/tmpg6p6qect/model/data/model/assets


INFO:tensorflow:Assets written to: /var/folders/m8/0_prp1tj41s9n5xm0bfqp6wm0000gn/T/tmpg6p6qect/model/data/model/assets


# Train/Val Workflow

Now to use the above workflow in hyperparameter tuning experiments, I'm going to functionalize it below.  The inputs include the following: 
- run_name
- training data
- validation data
- model
- hyperparameters (for logging)

The function should output a measure of loss to pass to optimization.

In [16]:
def run_log_exp(
    run_name: str,
    train_data: tuple,
    val_data: tuple,
    model: keras.Model,
    hyper_params: dict,
):
    with mlflow.start_run(run_name=run_name):
        history = model.fit(
            train_data[0],
            train_data[1],
            epochs=250,
            validation_data=val_data,
            callbacks=[
                keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
            ],
            verbose=0,
        )
        mlflow.log_params(hyper_params)
        # customization needed here depending on output shape
        total_loss, __, __ = model.evaluate(
            val_data[0], val_data[1], verbose=0
        )  # get validation loss for optimization
        return total_loss



In [15]:
mlflow.set_experiment(experiment_name="exp-2")

train_data = (X_train, {"main_output": y_train, "aux_output": y_train})
val_data = (X_valid, {"main_output": y_valid, "aux_output": y_valid})
hyper_params = {"n_hidden": 1, "n_neurons": 10, "activation": "relu"}
model = build_model(**hyper_params)
val_loss = run_log_exp(
    run_name="run-1",
    train_data=train_data,
    val_data=val_data,
    model=model,
    hyper_params=hyper_params,
)



1/1 [==============================] - 0s 71ms/step


2023/02/15 10:29:28 WARNING mlflow.tensorflow: Failed to infer model signature: Expected one of (pandas.DataFrame, numpy array, dictionary of (name -> numpy.ndarray), pyspark.sql.DataFrame) but got '<class 'list'>'
2023/02/15 10:29:28 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: /var/folders/m8/0_prp1tj41s9n5xm0bfqp6wm0000gn/T/tmpzy18cfmc/model/data/model/assets


INFO:tensorflow:Assets written to: /var/folders/m8/0_prp1tj41s9n5xm0bfqp6wm0000gn/T/tmpzy18cfmc/model/data/model/assets


104/104 [==============================] - 0s 2ms/step - loss: 0.3715 - main_output_loss: 0.3684 - aux_output_loss: 0.4002


## Running Experiments

Now I can run any number of experiments by calling the above function inside a looped grid of hyperparameters or inside an optimization function.  Below, I'll execute a small grid as an example.

In [19]:
for n_h in [1, 2, 3]:
    for n_n in [10, 20, 30]:
        for act in ["sigmoid", "relu"]:
            hyper_params = {"n_hidden": n_h, "n_neurons": n_n, "activation": act}
            run_name = f"run-{n_h}-{n_n}-{act}"
            model = build_model(**hyper_params)
            val_loss = run_log_exp(
                run_name=run_name,
                train_data=train_data,
                val_data=val_data,
                model=model,
                hyper_params=hyper_params,
            )
            print(f"validation loss for {run_name}: {round(val_loss, 4)}")


1/1 [==============================] - 0s 329ms/step


2023/02/15 10:34:48 WARNING mlflow.tensorflow: Failed to infer model signature: Expected one of (pandas.DataFrame, numpy array, dictionary of (name -> numpy.ndarray), pyspark.sql.DataFrame) but got '<class 'list'>'
2023/02/15 10:34:48 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: /var/folders/m8/0_prp1tj41s9n5xm0bfqp6wm0000gn/T/tmpn7haswxv/model/data/model/assets


INFO:tensorflow:Assets written to: /var/folders/m8/0_prp1tj41s9n5xm0bfqp6wm0000gn/T/tmpn7haswxv/model/data/model/assets


validation loss for run-1-10-sigmoid: 0.4675
1/1 [==============================] - 0s 55ms/step


...

validation loss for run-3-30-sigmoid: 0.6158
1/1 [==============================] - 0s 76ms/step


2023/02/15 11:02:06 WARNING mlflow.tensorflow: Failed to infer model signature: Expected one of (pandas.DataFrame, numpy array, dictionary of (name -> numpy.ndarray), pyspark.sql.DataFrame) but got '<class 'list'>'
2023/02/15 11:02:06 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: /var/folders/m8/0_prp1tj41s9n5xm0bfqp6wm0000gn/T/tmp2ozhcsg2/model/data/model/assets


INFO:tensorflow:Assets written to: /var/folders/m8/0_prp1tj41s9n5xm0bfqp6wm0000gn/T/tmp2ozhcsg2/model/data/model/assets


validation loss for run-3-30-relu: 0.323


# Cross Fold Validation

Cross fold validation is the gold standard for model candidate evaluation, however it's usually not implemented for DL models due to the model complexity and computational constraints.  If the models and/or data are small enough, however, it can still be done.  Here, I'm going to line out a work flow for applying CV and logging within MLFlow.

## Modify Fit and Validate

First, I'll need to modify the keras fit and validation strategy above, before adding the MLFlow logging.  I'll use `sklearn.model_selection.KFold()` to make the splits.  Since I no longer need a static validation set, I'll start from `X_train_full` and `y_train_full` from above.

### Reload Full Training Set

In [25]:
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target, test_size=0.2
)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train_full)
X_test = scaler.transform(X_test)
y_train = y_train_full  # for consistency in naming

pd.DataFrame(X_train, columns=housing.feature_names).describe()


,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
count,1.651200e+04,1.651200e+04,1.651200e+04,1.651200e+04,1.651200e+04,1.651200e+04,1.651200e+04,1.651200e+04
mean,-1.324888e-14,-1.265676e-16,-1.246365e-15,9.997655e-16,-4.927153e-17,1.964729e-15,2.133360e-14,-9.122580e-14
std,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00,1.000030e+00
min,-1.775581e+00,-2.191347e+00,-1.790362e+00,-1.542747e+00,-1.244706e+00,-1.034803e+00,-1.441052e+00,-2.367125e+00
25%,-6.894249e-01,-8.405722e-01,-3.884574e-01,-1.862380e-01,-5.584437e-01,-2.425336e-01,-7.908917e-01,-1.107928e+00
50%,-1.752218e-01,3.345851e-02,-8.172164e-02,-1.004821e-01,-2.279887e-01,-6.753845e-02,-6.412145e-01,5.360240e-01
75%,4.637940e-01,6.691172e-01,2.431791e-01,1.603766e-03,2.609186e-01,1.432883e-01,9.771707e-01,7.758711e-01
max,5.885792e+00,1.860977e+00,5.321753e+01,6.640870e+01,2.994658e+01,1.033827e+02,2.955717e+00,2.624692e+00


### Manual K-Fold CV

As I loop through each fold I'll log train and validation CV scores so I have access to the individual values as well as the aggregate.

In [27]:
from sklearn.model_selection import KFold
import numpy as np

mlflow.tensorflow.autolog(disable=True)  # turn off logging

hyper_params = {"n_hidden": 2, "n_neurons": 15, "activation": "relu"}

kf = KFold(
    n_splits=5, shuffle=True, random_state=629
)  # define random for reproducibility
cv_train_losses = []
cv_val_losses = []
for fit, val in kf.split(X_train, y_train):
    # define train and validation set per fold
    train_data = (
        X_train[fit],
        {"main_output": y_train[fit], "aux_output": y_train[fit]},
    )
    val_data = (X_train[val], {"main_output": y_train[val], "aux_output": y_train[val]})
    model = build_model(**hyper_params)
    history = model.fit(
        train_data[0],
        train_data[1],
        epochs=20,
        validation_data=val_data,
        callbacks=[
            keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
        ],
        verbose=0,
    )
    # update per fold loss
    train_loss, __, __ = model.evaluate(train_data[0], train_data[1], verbose=0)
    val_loss, __, __ = model.evaluate(val_data[0], val_data[1], verbose=0)
    cv_train_losses.append(train_loss)
    cv_val_losses.append(val_loss)

print(f"mean train mse: {round(np.mean(cv_train_losses), 4)}")
print(f"mean validation mse: {round(np.mean(cv_val_losses), 4)}")


mean train mse: 0.4486
mean validation mse: 0.4591


## MLFlow Logging Each Fold

Now to modify the MLFlow logging piece, I'll add nested runs for each fold.  In this way, the autologger will log all relevant information for each model trained on each fold, then in the parent run I'll manually log the aggregated loss values along with the chosen hyperparameters.  This will let me identify promising hyperparameter combinations by sorting through the aggregated metrics, but also still allow me to deep dive into any one experiment and view metrics and learning curves of each fold.  The nesting is handled by `nested=True` within `mlflow.start_run()`.

In [33]:
mlflow.tensorflow.autolog()  # turn on logging
mlflow.set_experiment(experiment_name="cv-test")

run_name = "test-cv"
with mlflow.start_run(run_name=run_name):
    cv_train_losses = []
    cv_val_losses = []
    k_fold = 1  # keep track of fold number
    for fit, val in kf.split(X_train, y_train):
        with mlflow.start_run(run_name=f"f{k_fold}-{run_name}", nested=True):
            # define train and validation set per fold
            train_data = (
                X_train[fit],
                {"main_output": y_train[fit], "aux_output": y_train[fit]},
            )
            val_data = (
                X_train[val],
                {"main_output": y_train[val], "aux_output": y_train[val]},
            )
            model = build_model(**hyper_params)
            history = model.fit(
                train_data[0],
                train_data[1],
                epochs=20,
                validation_data=val_data,
                callbacks=[
                    keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
                ],
                verbose=0,
            )
            mlflow.log_params(hyper_params)  # log chosen params in child
            # update per fold loss
            train_loss, __, __ = model.evaluate(train_data[0], train_data[1], verbose=0)
            val_loss, __, __ = model.evaluate(val_data[0], val_data[1], verbose=0)
            cv_train_losses.append(train_loss)
            cv_val_losses.append(val_loss)
            k_fold += 1  # update fold number
    mlflow.log_params(hyper_params)  # log chosen params in parent
    # log aggregated metrics in parent
    mlflow.log_metrics(
        {
            "train_mean_cv_loss": np.mean(cv_train_losses),
            "val_mean_cv_loss": np.mean(cv_val_losses),
        }
    )



1/1 [==============================] - 0s 65ms/step


2023/02/15 12:12:20 WARNING mlflow.tensorflow: Failed to infer model signature: Expected one of (pandas.DataFrame, numpy array, dictionary of (name -> numpy.ndarray), pyspark.sql.DataFrame) but got '<class 'list'>'
2023/02/15 12:12:20 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: /var/folders/m8/0_prp1tj41s9n5xm0bfqp6wm0000gn/T/tmpe_ex01bo/model/data/model/assets


INFO:tensorflow:Assets written to: /var/folders/m8/0_prp1tj41s9n5xm0bfqp6wm0000gn/T/tmpe_ex01bo/model/data/model/assets


1/1 [==============================] - 0s 64ms/step


...

INFO:tensorflow:Assets written to: /var/folders/m8/0_prp1tj41s9n5xm0bfqp6wm0000gn/T/tmp67yv8q0p/model/data/model/assets


1/1 [==============================] - 0s 68ms/step


2023/02/15 12:14:01 WARNING mlflow.tensorflow: Failed to infer model signature: Expected one of (pandas.DataFrame, numpy array, dictionary of (name -> numpy.ndarray), pyspark.sql.DataFrame) but got '<class 'list'>'
2023/02/15 12:14:01 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: /var/folders/m8/0_prp1tj41s9n5xm0bfqp6wm0000gn/T/tmplttvy9ml/model/data/model/assets


INFO:tensorflow:Assets written to: /var/folders/m8/0_prp1tj41s9n5xm0bfqp6wm0000gn/T/tmplttvy9ml/model/data/model/assets
